In [ ]:
!pip install pynvml

!pip install zeus[bso]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.2
    Uninstalling pydantic-2.11.2:
      Successfully uninstalled pydantic-2.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.23 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.21 which is incompatible.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 1.10.21 which is incompatible.
thinc 8.3.6 requires pydantic<3.0.0,>=2.0.0, but you have pyda

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import math
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset
from pynvml import *
#from codecarbon import EmissionsTracker
import torch.profiler
from sklearn.metrics import mean_absolute_percentage_error
import time
from zeus.optimizer.power_limit import GlobalPowerLimitOptimizer
from zeus.monitor import ZeusMonitor
from zeus.optimizer.batch_size import BatchSizeOptimizer, JobSpec
import csv
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction
from torch.utils.data import Dataset


Mounted at /content/drive
/opt/rocm/lib/libamd_smi.so: cannot open shared object file: No such file or directory
Unable to find libamd_smi.so library try installing amd-smi-lib from your package manager


In [ ]:
import os
os.environ["TORCH_MHA_DISABLE_EFFICIENT"] = "1"  # Force fallback to standard attention kernel
print("PyTorch version:", torch.__version__)
from sklearn.preprocessing import StandardScaler
#tracker.stop()

PyTorch version: 2.6.0+cu124


In [ ]:
batch_opt = False
optimize = False
freq_opt = False

In [ ]:
# def create_sliding_windows(df, input_window=168, forecast_horizon=24, target_column='Settlement Point Price'):
#     X, y = [], []
#     df = df.sort_index()
#     for i in range(len(df) - input_window - forecast_horizon + 1):
#         X_window = df.iloc[i:i+input_window].drop(target_column, axis=1).values
#         y_window = df[target_column].iloc[i+input_window:i+input_window+forecast_horizon].values
#         X.append(X_window)
#         y.append(y_window)
#     return np.array(X), np.array(y)



def create_sliding_windows_with_future_weather(df, input_window=24, forecast_horizon=24,
                                               target_column='Settlement Point Price',
                                               weather_columns=['Air Temp (°C)', 'Wind Speed (m/s)']):

    X, y = [], []
    df = df.sort_index()

    # Get the list (Index) of feature columns (all columns except target)
    full_feature_columns = df.columns.drop(target_column)
    n_features = len(full_feature_columns)

    for i in range(len(df) - input_window - forecast_horizon + 1):
        # Historical window: use all features (except target)
        X_hist = df.iloc[i : i + input_window].drop(target_column, axis=1).values  # shape (input_window, n_features)

        # Future window: create an array of zeros with the same number of features
        X_future_full = np.zeros((forecast_horizon, n_features))

        # Get the forecast weather values and fill into the corresponding columns
        future_df = df.iloc[i + input_window : i + input_window + forecast_horizon]
        for col in weather_columns:
            if col in full_feature_columns:
                # Find the index of the weather column in the feature set
                idx = full_feature_columns.get_loc(col)
                X_future_full[:, idx] = future_df[col].values
            else:
                raise ValueError(f"Column '{col}' not found in dataframe columns: {list(full_feature_columns)}")

        # Concatenate along the time axis to form the combined input window
        X_combined = np.concatenate([X_hist, X_future_full], axis=0)

        # The target is the forecast horizon values for the target column.
        y_window = df[target_column].iloc[i + input_window : i + input_window + forecast_horizon].values

        X.append(X_combined)
        y.append(y_window)

    return np.array(X), np.array(y)

In [ ]:
#inspo: https://stackoverflow.com/questions/77444485/using-positional-encoding-in-pytorch

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)



class PositionalEncodingExo(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Create positional encoding matrix once
        position = torch.arange(0, max_len).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)  # makes it non-trainable

    def forward(self, x):
        """
        x: Tensor of shape (seq_len, batch_size, d_model)
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerExogenousForecastModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, seq_len, forecast_horizon, dropout=0.1):
        super().__init__()
        self.seq_len = seq_len
        self.forecast_horizon = forecast_horizon

        # Project raw features to d_model
        self.input_proj = nn.Linear(input_dim, d_model)

        # Positional encoding
        self.pos_encoder = PositionalEncodingExo(d_model, dropout, max_len=seq_len)

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=4 * d_model, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Final output head
        self.fc_out = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, forecast_horizon)
        )

    def forward(self, x):
        """
        Args:
            x: Tensor of shape (batch_size, seq_len, input_dim)
        Returns:
            Tensor of shape (batch_size, forecast_horizon)
        """
        x = self.input_proj(x)           # (B, T, d_model)
        x = x.transpose(0, 1)            # (T, B, d_model)
        x = self.pos_encoder(x)          # (T, B, d_model)
        x = self.transformer_encoder(x)  # (T, B, d_model)
        x = x.transpose(0, 1)            # (B, T, d_model)

        # Mean pool over time
        pooled = x.mean(dim=1)           # (B, d_model)
        return self.fc_out(pooled)       # (B, forecast_horizon)

class TransformerSeq2SeqForecastModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_encoder_layers, num_decoder_layers,
                 seq_len, forecast_horizon, dropout=0.1):
        super().__init__()
        dim_feedforward = 4 * d_model
        self.input_proj = nn.Linear(input_dim, d_model)
        self.tgt_proj = nn.Linear(1, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len=seq_len)
        self.pos_decoder = PositionalEncoding(d_model, dropout, max_len=forecast_horizon)
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_decoder_layers)
        self.fc_out = nn.Linear(d_model, 1)
        self.seq_len = seq_len
        self.forecast_horizon = forecast_horizon

    def forward(self, src_inputs, tgt=None):
        """
        Args:
            src_inputs: A list containing:
                - src_inputs[0]: historical sequence Tensor of shape (B, seq_len, input_dim)
                - src_inputs[1]: future time encoding Tensor.
            tgt: Optional target Tensor for teacher forcing.
        Returns:
            Tensor of shape (B, forecast_horizon) with forecasted values.
        """

        batch_size = src.size(0)

        # === ENCODER ===
        src_emb = self.input_proj(src)  # (B, seq_len, d_model)
        src_emb = self.pos_encoder(src_emb.transpose(0, 1))  # (seq_len, B, d_model)
        memory = self.transformer_encoder(src_emb)  # (seq_len, B, d_model)

        # === DECODER ===
        if tgt is not None:
            # Teacher forcing mode:
            if tgt.dim() == 2:
                tgt = tgt.unsqueeze(-1)  # (B, H, 1)
            decoder_input = tgt[:, :-1, :]  # (B, H-1, input_dim)
            start_token = torch.zeros(batch_size, 1, tgt.size(-1), device=tgt.device)
            decoder_input = torch.cat([start_token, decoder_input], dim=1)  # (B, H, input_dim)
            decoder_input = self.tgt_proj(decoder_input)  # (B, H, d_model)
            decoder_input = decoder_input.transpose(0, 1)  # (H, B, d_model)
            decoder_input = self.pos_decoder(decoder_input)
            tgt_len = decoder_input.size(0)
            tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_len).to(tgt.device)
            decoder_output = self.transformer_decoder(decoder_input, memory, tgt_mask=tgt_mask)
            output = self.fc_out(decoder_output)  # (H, B, 1)
            output = output.transpose(0, 1).squeeze(-1)  # (B, H)
        else:
            # Autoregressive inference (no teacher forcing)
            outputs = []
            prev_y = torch.zeros(batch_size, 1, device=src.device)
            for t in range(self.forecast_horizon):
                dec_input_step = self.tgt_proj(prev_y.unsqueeze(-1))  # (B, 1, d_model)
                dec_input_step = dec_input_step.transpose(0, 1)  # (1, B, d_model)
                dec_input_step = self.pos_decoder(dec_input_step)
                tgt_mask = nn.Transformer.generate_square_subsequent_mask(dec_input_step.size(0)).to(src.device)
                dec_output = self.transformer_decoder(dec_input_step, memory, tgt_mask=tgt_mask)
                out_step = self.fc_out(dec_output)  # (1, B, 1)
                out_step = out_step.squeeze(0).squeeze(-1)  # (B,)
                outputs.append(out_step)
                prev_y = out_step.unsqueeze(1)
            output = torch.stack(outputs, dim=1)  # (B, H)
        return output




In [ ]:
# config = TimeSeriesTransformerConfig(
#     feature_size=11,            # number of features per timestep
#     hidden_size=128,            # embedding dimension for the transformer
#     num_hidden_layers=4,        # number of encoder layers
#     num_attention_heads=8,      # number of attention heads
#     dropout=0.1,                # dropout probability
#     layer_norm_eps=1e-5,        # layer norm epsilon
#     initializer_range=0.02,     # weight initialization range
# )

# model = TimeSeriesTransformerModel(config)

In [ ]:

df = pd.read_csv('ERCOT_Data/Processed/Houston_Load_SPP_Weather_Combined_updated.csv')
df_unscaled = pd.read_csv('ERCOT_Data/Processed/Houston_Load_SPP_Weather_Combined_updated.csv')

df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.sort_values('Datetime').reset_index(drop=True)

# Extract hour,day,month features
df['hour'] = df['Datetime'].dt.hour
df['dayofweek'] = df['Datetime'].dt.dayofweek
df['month'] = df['Datetime'].dt.month

# Cyclic encoding
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['dow_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# imputing NAN vals (i figure it's fine because there are so few)
df['Air Temp (°C)'] = df['Air Temp (°C)'].interpolate(method='linear').ffill().bfill()
df['Wind Speed (m/s)'] = df['Wind Speed (m/s)'].interpolate(method='linear').ffill().bfill()



#defing features and target

feature_cols = [
    'Settlement Point Price',
    #'Houston_Load',
    'Wind Speed (m/s)', 'Air Temp (°C)',
    'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos'
]
time_encoding_cols = ['hour_sin', 'hour_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos']

y_col = 'Settlement Point Price'


# Parameters
lookback_window = 24
forecast_horizon = 24

# Define test set as last 5%
test_size = int(0.10 * len(df))
test_cutoff_index = len(df) - test_size
df_trainval = df.iloc[:test_cutoff_index].copy()
df_test = df.iloc[test_cutoff_index - lookback_window:].copy()  # ensure overlap for lookback


In [ ]:
print("Hour range:", df['hour'].min(), "→", df['hour'].max())

print(df[['Datetime']].head())
print(df[['Datetime']].tail())


Hour range: 0 → 23
             Datetime
0 2021-04-15 13:00:00
1 2021-04-15 14:00:00
2 2021-04-15 15:00:00
3 2021-04-15 16:00:00
4 2021-04-15 17:00:00
                 Datetime
29264 2024-12-31 19:00:00
29265 2024-12-31 20:00:00
29266 2024-12-31 21:00:00
29267 2024-12-31 22:00:00
29268 2024-12-31 23:00:00


In [ ]:
# Scale
scaler_target = StandardScaler()
scaler_weather = StandardScaler()
scaler_target.fit(df_trainval[[y_col]])
scaler_weather.fit(df_trainval[['Air Temp (°C)', 'Wind Speed (m/s)']])

df_trainval[y_col] = scaler_target.transform(df_trainval[[y_col]])
df_trainval[['Air Temp (°C)', 'Wind Speed (m/s)']] = scaler_weather.transform(
df_trainval[['Air Temp (°C)', 'Wind Speed (m/s)']])

df_test_scaled = df_test.copy()
df_test_scaled[y_col] = scaler_target.transform(df_test_scaled[[y_col]])
df_test_scaled[['Air Temp (°C)', 'Wind Speed (m/s)']] = scaler_weather.transform(
df_test_scaled[['Air Temp (°C)', 'Wind Speed (m/s)']])

# Sliding window function
def create_sliding_windows_with_future_time(df, lookback, horizon, feature_cols, time_encoding_cols, target_col):
    x_list, time_future_list, y_list = [], [], []
    data = df[feature_cols].values
    time_enc = df[time_encoding_cols].values
    target = df[[target_col]].values
    for t in range(lookback, len(df) - horizon):
        x_seq = data[t - lookback:t]
        future_time = time_enc[t:t + horizon]
        y = target[t:t + horizon].flatten()
        x_list.append(x_seq)
        time_future_list.append(future_time)
        y_list.append(y)
    return np.array(x_list), np.array(time_future_list), np.array(y_list)

In [ ]:
# Train/val sliding windows
x_seq, time_enc_seq, y = create_sliding_windows_with_future_time(
    df_trainval, lookback_window, forecast_horizon, feature_cols, time_encoding_cols, y_col)
print("x_seq_train shape:", np.shape(x_seq))
print("time_enc_train shape:", np.shape(time_enc_seq))
print("y_train shape:", np.shape(y))
d_model= 256
nhead = 8
num_layers = 4
seq_len = 24
dr = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerSeq2SeqForecastModel(15, d_model, nhead, num_layers, num_layers, seq_len, forecast_horizon, dropout = dr).to(device)

from sklearn.model_selection import train_test_split
random_state = 42


# Shuffle and split train/val
np.random.seed(42)
indices = np.arange(len(x_seq))
np.random.shuffle(indices)
x_seq = x_seq[indices]
time_enc_seq = time_enc_seq[indices]
y = y[indices]

split_idx = int(len(x_seq) * 0.9)
x_seq_train, x_seq_val = x_seq[:split_idx], x_seq[split_idx:]
time_enc_train, time_enc_val = time_enc_seq[:split_idx], time_enc_seq[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

class TimeSeriesDataset(Dataset):
    def __init__(self, inputs, y_data):
        """
        Args:
            inputs: a list or tuple with two elements:
                inputs[0]: historical sequence data (e.g., x_seq)
                inputs[1]: future time encoding data (e.g., time_enc)
            y_data: target values.
        """
        self.x_data = torch.tensor(inputs[0], dtype=torch.float32)
        self.time_data = torch.tensor(inputs[1], dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        # Return a tuple:
        #   First element: a list containing the two input tensors.
        #   Second element: the target tensor.
        return [self.x_data[idx], self.time_data[idx]], self.y_data[idx]

# Create dataset objects for training and validation.
train_dataset = TimeSeriesDataset([x_seq_train, time_enc_train], y_train)
val_dataset   = TimeSeriesDataset([x_seq_val, time_enc_val], y_val)

# Define DataLoader objects for training and validation.
# Adjust batch_size and num_workers as needed.
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)


x_seq_train shape: (26295, 24, 9)
time_enc_train shape: (26295, 24, 6)
y_train shape: (26295, 24)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
from tqdm import tqdm
# Define the optimizer and loss function.
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.HuberLoss()
num_epochs = 100
#tracker.start()
# Training loop
best_val_loss = float('inf')
monitor_b = False
if monitor_b:

  monitor = ZeusMonitor(gpu_indices=[torch.cuda.current_device()])


optimal_batch_size = 64

if batch_opt:
  job_spec = JobSpec(
    job_id=os.environ.get("ZEUS_JOB_ID"),
    job_id_prefix="transformerEnergyForece",
    default_batch_size=64,
    batch_sizes=[16, 32, 64, 128, 256, 512],  # Search space
    max_epochs=10,
    target_val_metric=0.05,
    higher_metric_is_better=False  # lower MSE is better
    )
  server_url = "http://bso-server:8000"  # or whatever the Zeus server address is

  bso = BatchSizeOptimizer(
    monitor=monitor,
    server_url=server_url,
    job=job_spec,
    rank=0  # rank=0 for non-distributed training
  )


  optimal_batch_size = bso.get_batch_size()

  print(f"Optimal batch size found: {optimal_batch_size}")




plo = None
val_losses = []
train_losses = []  # Initialize an empty list for training losses
real_val_mae= []
if(optimize):
  plo = GlobalPowerLimitOptimizer(monitor)

scheduler = None
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.75, patience=3, min_lr=1e-6)

epoch_energy = []
avg_steps = []
print(f"Train power optimized: {int(optimize)} batch optimized: {int(batch_opt)} freq optimized: {int(freq_opt)}")


if batch_opt:
  bso.on_train_begin()


initial_tf_ratio = 1
final_tf_ratio = 0.2
tf_ratio = []
warmup = 3
slow = 20
decay_rate = 0.03
slow_decay = 0.01
for epoch in range(num_epochs):
  if(epoch <warmup):
    teacher_forcing_ratio = 1
  elif(epoch < slow):
    ratio = initial_tf_ratio * math.exp(-slow_decay * (epoch-warmup))
    teacher_forcing_ratio = max(ratio, final_tf_ratio)
  else:
    ratio = initial_tf_ratio * math.exp(-decay_rate * (epoch-slow))
    teacher_forcing_ratio = max(ratio, final_tf_ratio)

  tf_ratio.append(teacher_forcing_ratio)
print(tf_ratio)
last_lr = optimizer.param_groups[0]['lr']
weights = torch.linspace(0.1, 1.0, steps=forecast_horizon).to(device) #incase we want to do smoothing

Train power optimized: 0 batch optimized: 0 freq optimized: 0
[1, 1, 1, 1.0, 0.9900498337491681, 0.9801986733067553, 0.9704455335485082, 0.9607894391523232, 0.951229424500714, 0.9417645335842487, 0.9323938199059483, 0.9231163463866358, 0.9139311852712282, 0.9048374180359595, 0.8958341352965282, 0.8869204367171575, 0.8780954309205613, 0.8693582353988059, 0.8607079764250578, 0.8521437889662113, 1.0, 0.9704455335485082, 0.9417645335842487, 0.9139311852712282, 0.8869204367171575, 0.8607079764250578, 0.835270211411272, 0.8105842459701871, 0.7866278610665535, 0.7633794943368531, 0.7408182206817179, 0.7189237334319262, 0.697676326071031, 0.6770568744981647, 0.6570468198150567, 0.6376281516217733, 0.6187833918061408, 0.6004955788122659, 0.5827482523739896, 0.5655254386995371, 0.5488116360940264, 0.5325918010068972, 0.5168513344916993, 0.5015760690660556, 0.4867522559599717, 0.4723665527410147, 0.4584060113052235, 0.44485806622294116, 0.43171052342907973, 0.418951549247639, 0.40656965974059917,

In [ ]:
print(f"train for window {seq_len} d_model {d_model} nhead {nhead} depth {num_layers} batch size {optimal_batch_size} with scheduler {scheduler != None} and initial lr {last_lr}")

for epoch in range(15):
    teacher_forcing_ratio = tf_ratio[epoch]
    model.train()
    train_loss = 0.0
    # if monitor_b:
    #   monitor.begin_window("epoch")  # start epoch-level energy/time window
    #   steps = []
    # if(optimize):
    #   plo.on_epoch_begin()

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    for batch_inputs, batch_y in pbar:
    #   if optimize:
    #       plo.on_step_begin()

    #   if monitor_b:
    #       monitor.begin_window("step")  # start step-level window


      batch_x, batch_time = batch_inputs
      batch_x = batch_x.to(device)
      batch_time = batch_time.to(device)
      batch_y = batch_y.to(device)

      # For Option 1:
      # Combine along last dimension:
      src = torch.cat([batch_x, batch_time], dim=-1)

      tgt_input = batch_y.unsqueeze(-1) + torch.randn_like(batch_y.unsqueeze(-1)) * 0.01
      use_tf = (random.random() < teacher_forcing_ratio)
      if use_tf:
          outputs = model(src, tgt_input)
      else:
          outputs = model(src, None)

      loss = criterion(outputs, batch_y)  # (batch_size, forecast_horizon)
      #loss = (loss * weights).mean() # incase want to do weight smoothing

      optimizer.zero_grad()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      optimizer.step()

      train_loss += loss.item() * src.size(0)

      # End the step window and record its metrics.
      # if monitor_b:
      #     step_result = monitor.end_window("step")
      #     steps.append(step_result)

      # #Let the global power optimizer examine this step
      # if optimize:
      #    plo.on_step_end()

    train_loss /= len(train_dataset)
    # if monitor_b:
    #   mes = monitor.end_window("epoch")  # complete the epoch window
    #   epoch_energy.append(mes)
    # if optimize:
    #   plo.on_epoch_end()


    # if monitor_b:

    #   print(f"Epoch {epoch} consumed {mes.time} s and {mes.total_energy} J.")

    #   avg_time = sum(m.time for m in steps) / len(steps)
    #   avg_energy = sum(m.total_energy for m in steps) / len(steps)
    #   print(f"One step took {avg_time:.4f} s and {avg_energy:.4f} J on average.")

    #   avg_steps.append([avg_time, avg_energy])

    # Evaluate on validation set
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_targets = []
    with torch.no_grad():
      for batch_inputs, batch_y in val_loader:
          batch_x, batch_time = batch_inputs
          batch_x = batch_x.to(device)
          batch_time = batch_time.to(device)
          batch_y = batch_y.to(device)

          # For Option 1:
          # Combine along last dimension:
          src = torch.cat([batch_x, batch_time], dim=-1)


          outputs = model(src,None)

          loss = criterion(outputs, batch_y)
          val_loss += loss.item() * src.size(0)



      val_loss /= len(val_dataset)

      # if scheduler != None:
      #   curr_lr = scheduler.get_last_lr()[0]
      #   if curr_lr != last_lr:
      #     last_lr = curr_lr
      #     print(f"New learning rate: {curr_lr}")
      #   scheduler.step(val_loss)


      print(f"TF ratio: {teacher_forcing_ratio}, Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

      train_losses.append(train_loss)
      val_losses.append(val_loss)

    # if batch_opt:

    #   bso.on_evaluate(val_loss)

    #   # The BSO server will determine whether to stop training
    #   if bso.training_finished:
    #     break

    # Save the model if validation loss has improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"best_transformer_model_window_{seq_len}_d_{d_model}_nhead_{nhead}_depth_{num_layers}_droput_{dr}_batchsize_{optimal_batch_size}.pth")
        print("New best model saved!")

with open(f"Energy Consumption Data/OPTIMIZER_epoch_energy_transformer_optimized_{int(optimize)}_batch_{int(batch_opt)}_freq_{int(freq_opt)}.csv", "w", newline="") as csvfile:
    fieldnames = ["epoch", "time_seconds", "energy_joules", "avg_step_energy", "avg_step_time", "Val MSE"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    i = 0
    for epoch, mes in enumerate(epoch_energy):
        writer.writerow({
            "epoch": epoch,
            "time_seconds": mes.time,          # assumes mes.time holds time in seconds
            "energy_joules": mes.total_energy,  # assumes mes.total_energy holds energy in Joules
            "avg_step_energy": avg_steps[i][1],
            "avg_step_time": avg_steps[i][0],
            "Val MSE": val_losses[i]
        })
        i+=1

train for window 24 d_model 256 nhead 8 depth 4 batch size 64 with scheduler False and initial lr 0.0005


Epoch 1/100: 100%|██████████| 370/370 [00:10<00:00, 34.22batch/s]


TF ratio: 1, Epoch 1/100, Train Loss: 0.0555, Val Loss: 0.1060


Epoch 2/100: 100%|██████████| 370/370 [00:11<00:00, 33.51batch/s]


TF ratio: 1, Epoch 2/100, Train Loss: 0.0488, Val Loss: 0.1473


...

TF ratio: 1.0, Epoch 4/100, Train Loss: 0.0445, Val Loss: 0.1106


Epoch 5/100: 100%|██████████| 370/370 [00:11<00:00, 31.31batch/s]


TF ratio: 0.9900498337491681, Epoch 5/100, Train Loss: 0.0486, Val Loss: 0.1058


Epoch 6/100:  33%|███▎      | 123/370 [00:04<00:09, 26.55batch/s]


KeyboardInterrupt: 

In [ ]:
epochs = range(1, len(train_losses) + 1)

fig, ax1 = plt.subplots()
plt.plot(epochs, train_losses, 'b-o', label='Train Loss')
plt.plot(epochs, val_losses, 'r-o', label='Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('MAE Loss')
ax2 = ax1.twinx()
ax2.plot(epochs, tf_ratio[0:len(train_losses)], 'g-o', label='Teacher Forcing Ratio')
ax2.set_ylabel('Teacher Forcing Ratio')
ax1.legend()
plt.title('Train vs. Validation Loss with tf Ratio on left')
ax2.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
predictions = []
actuals = []

from sklearn.metrics import mean_absolute_error

with torch.no_grad():
      for batch_inputs, batch_y in val_loader:
          batch_x, batch_time = batch_inputs
          batch_x = batch_x.to(device)
          batch_time = batch_time.to(device)
          batch_y = batch_y.to(device)

          # For Option 1:
          # Combine along last dimension:
          src = torch.cat([batch_x, batch_time], dim=-1)


          outputs = model(src,None)

          loss = criterion(outputs, batch_y)
          val_loss += loss.item() * src.size(0)

          predictions.append(outputs.cpu().numpy())
          actuals.append(batch_y.cpu().numpy())

predictions = np.concatenate(predictions, axis=0)  # (num_val_samples, forecast_horizon)

actuals = np.concatenate(actuals, axis=0)          # (num_val_samples, forecast_horizon)
#predictions = np.concatenate(predictions, axis=0)  # (num_val_samples, forecast_horizon) # This line was causing issues and is not needed

# Reshape predictions_unscaled to match actuals shape before calculating RMSE
predictions_unscaled = scaler_target.inverse_transform(predictions.reshape(-1, 1)).reshape(actuals.shape)
actuals_unscaled = scaler_target.inverse_transform(actuals.reshape(-1, 1)).reshape(actuals.shape) # Assuming actuals also needs unscaling

sample_idx = random.randint(0, len(val_dataset)-1)
forecast_horizon = predictions_unscaled.shape[1]

forecast_sample = predictions_unscaled[-1]  # shape (24,)

# Create an x-axis for the 24 forecast hours: 0..23
hours_future = np.arange(0, forecast_horizon)

# Calculate RMSE using the reshaped predictions_unscaled
rmse = np.sqrt(np.mean((predictions_unscaled - actuals) ** 2))
mae = mean_absolute_error(predictions_unscaled, actuals)
print("Validation RMSE:", rmse)
print(mae)

# plt.figure(figsize=(10, 4))
# plt.plot(hours_future, forecast_sample, 'o-', label='Forecasted Price')
# plt.plot(hours_future, actuals[-1], 'o-', label='Actual Future Price')
# plt.title('24-Hour Price Forecast vs Actual')
# plt.xlabel('Hour Relative to Forecast Start')
# plt.ylabel('Settlement Point Price ($/MWh)')
# plt.grid(True)
# plt.legend()
# plt.tight_layout()
# plt.show()

# predictions = []
# actuals = []
# with torch.no_grad():
#     for batch_X, batch_y in train_loader:
#         batch_X = batch_X.to(device)
#         batch_y = batch_y.to(device)

#         outputs = model(batch_X)  # shape (batch_size, forecast_horizon)

#         # Move back to CPU and store
#         predictions.append(outputs.cpu().numpy())
#         actuals.append(batch_y.cpu().numpy())

# predictions = np.concatenate(predictions, axis=0)  # (num_val_samples, forecast_horizon)
# actuals = np.concatenate(actuals, axis=0)          # (num_val_samples, forecast_horizon)

# predictions_unscaled = scaler_y.inverse_transform(predictions)
# actuals_unscaled = scaler_y.inverse_transform(actuals)

# sample_idx = random.randint(0, len(val_dataset)-1)
# forecast_horizon = predictions_unscaled.shape[1]

# forecast_sample = predictions_unscaled[sample_idx]  # shape (24,)
# actual_sample   = actuals_unscaled[sample_idx]      # shape (24,)

# # Create an x-axis for the 24 forecast hours: 0..23
# hours_future = np.arange(0, forecast_horizon)

# plt.figure(figsize=(10, 4))
# plt.plot(hours_future, actual_sample, 'o-', label='Actual Future Price')
# plt.plot(hours_future, forecast_sample, 'o-', label='Forecasted Price')

# plt.title('24-Hour Price Forecast vs Actual for TRAIN SET')
# plt.xlabel('Hour Relative to Forecast Start')
# plt.ylabel('Settlement Point Price ($/MWh)')
# plt.grid(True)
# plt.legend()
# plt.tight_layout()
# plt.show()



Validation RMSE: 20.93526
26.755021005584716


In [ ]:
!pip install torchviz
!pip install hiddenlayer
!pip install torchinfo
!pip install graphviz

!pip install torchview


In [ ]:
from torchinfo import summary
from torchviz import make_dot
import io
import sys
from PIL import Image, ImageDraw, ImageFont
from torchview import draw_graph
model = TransformerSeq2SeqForecastModel(input_dim, d_model, nhead, num_layers, num_layers, seq_len, forecast_horizon).to_empty(device=device)

model.load_state_dict(torch.load(f"best_transformer_model_window_{seq_len}_d_{d_model}_nhead_{nhead}_depth_{num_layers}_droput_{dr}_batchsize_{optimal_batch_size}.pth"))

model_graph = draw_graph(model, input_size=[(64, 720, input_dim), (64, 24)], device='cuda')
model_graph.visual_graph

print(summary(model, input_size=[(64, 720, input_dim), (64, 24)]))


In [ ]:
from torchviz import make_dot

make_dot(yhat, params=dict(list(model.named_parameters()))).render("tx_torchviz", format="png")


# Transformer Works Poorly, seems like local variables like temp and wind speed are more important

In [ ]:
from tqdm import tqdm


In [ ]:
# Define hyperparameters
input_dim = len(df_transformed.columns)
seq_len = 24
forecast_horizon = 24
target_col_idx = df_transformed.columns.get_loc('Settlement Point Price')

X, y = create_sliding_windows_with_future_weather(df_transformed, input_window=seq_len, forecast_horizon=forecast_horizon, target_column='Settlement Point Price')


# Step 5: Train/Val split
split_idx = int(0.6 * len(X))
X_train, X_val = X[:split_idx], X[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

# Step 6: Normalize target (y)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)

# Step 7: Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32)

# Step 8: Build Datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Step 9: Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Debug prints
print("X_train:", X_train_tensor.shape, "y_train:", y_train_tensor.shape)
print("X_val:", X_val_tensor.shape, "y_val:", y_val_tensor.shape)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

def plot_losses(train_losses, val_losses, title="Training vs Validation Loss", save_path=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Train Loss", marker='o')
    plt.plot(val_losses, label="Validation Loss", marker='s')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(title)
    plt.legend()
    plt.grid(True)

    if save_path:
        plt.savefig(save_path)
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

def train_forecast_model(
    model,
    train_loader,
    val_loader,
    num_epochs,
    criterion,
    optimizer,
    device,
    scheduler=None,
    clip_grad=0.5,
    save_path="best_model_no_seq.pth"
):
    model.to(device)
    best_val_loss = float("inf")
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0.0

        for batch_X, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch_X = batch_X.to(device)  # (B, T, F)
            batch_y = batch_y.to(device)  # (B, H)

            optimizer.zero_grad()
            preds = model(batch_X)        # (B, H)

            loss = criterion(preds, batch_y)
            loss.backward()

            if clip_grad:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad)

            optimizer.step()
            epoch_train_loss += loss.item() * batch_X.size(0)

        epoch_train_loss /= len(train_loader.dataset)
        train_losses.append(epoch_train_loss)

        # ----------------- Validation -----------------
        model.eval()
        epoch_val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X = batch_X.to(device)
                batch_y = batch_y.to(device)
                preds = model(batch_X)
                loss = criterion(preds, batch_y)
                epoch_val_loss += loss.item() * batch_X.size(0)

        epoch_val_loss /= len(val_loader.dataset)
        val_losses.append(epoch_val_loss)

        # ----------------- Logging -----------------
        print(f"Epoch {epoch+1}/{num_epochs} — Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")

        # Save best model
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), save_path)
            print("New best model saved!")

        # Step the scheduler
        if scheduler is not None:
            scheduler.step(epoch_val_loss)

    plot_losses(train_losses, val_losses)

    return train_losses, val_losses


model = TransformerExogenousForecastModel(
    input_dim=11,
    d_model=256,
    nhead=8,
    num_layers=3,
    seq_len=24,
    forecast_horizon=24
)


X, y = create_sliding_windows_with_future_weather(combined_df, input_window=24, forecast_horizon=24, target_column='Settlement Point Price')


criterion = nn.HuberLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)
scheduler = None# torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5, min_lr=1e-6)

batch = next(iter(train_loader))
yhat = model(batch.text) # Give dummy batch to forward().
transforms = [ hl.transforms.Prune('Constant') ] # Removes Constant nodes from graph.

graph = hl.build_graph(model, batch.text, transforms=transforms)
graph.theme = hl.graph.THEMES['blue'].copy()
graph.save('tx_hiddenlayer', format='png')

make_dot(yhat, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")


train_losses, val_losses = train_forecast_model(
    model,
    train_loader,
    val_loader,
    num_epochs=20,
    criterion=criterion,
    optimizer=optimizer,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    scheduler=scheduler
)



In [ ]:
model.load_state_dict(torch.load(f"best_model_no_seq.pth"))


predictions = []
actuals = []

with torch.no_grad():
    for batch_X, batch_y in val_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_X)  # shape (batch_size, forecast_horizon)

        # Move back to CPU and store
        predictions.append(outputs.cpu().numpy())
        actuals.append(batch_y.cpu().numpy())

predictions = np.concatenate(predictions, axis=0)  # (num_val_samples, forecast_horizon)
actuals = np.concatenate(actuals, axis=0)          # (num_val_samples, forecast_horizon)

predictions_unscaled = scaler_y.inverse_transform(predictions)
actuals_unscaled = scaler_y.inverse_transform(actuals)

sample_idx = random.randint(0, len(val_dataset)-1)
forecast_horizon = predictions_unscaled.shape[1]

forecast_sample = predictions_unscaled[sample_idx]  # shape (24,)
actual_sample   = actuals_unscaled[sample_idx]      # shape (24,)

# Create an x-axis for the 24 forecast hours: 0..23
hours_future = np.arange(0, forecast_horizon)

plt.figure(figsize=(10, 4))
plt.plot(hours_future, actual_sample, 'o-', label='Actual Future Price')
plt.plot(hours_future, forecast_sample, 'o-', label='Forecasted Price')

plt.title('24-Hour Price Forecast vs Actual')
plt.xlabel('Hour Relative to Forecast Start')
plt.ylabel('Settlement Point Price ($/MWh)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

predictions = []
actuals = []
with torch.no_grad():
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_X)  # shape (batch_size, forecast_horizon)

        # Move back to CPU and store
        predictions.append(outputs.cpu().numpy())
        actuals.append(batch_y.cpu().numpy())

predictions = np.concatenate(predictions, axis=0)  # (num_val_samples, forecast_horizon)
actuals = np.concatenate(actuals, axis=0)          # (num_val_samples, forecast_horizon)

predictions_unscaled = scaler_y.inverse_transform(predictions)
actuals_unscaled = scaler_y.inverse_transform(actuals)

sample_idx = random.randint(0, len(val_dataset)-1)
forecast_horizon = predictions_unscaled.shape[1]

forecast_sample = predictions_unscaled[sample_idx]  # shape (24,)
actual_sample   = actuals_unscaled[sample_idx]      # shape (24,)

# Create an x-axis for the 24 forecast hours: 0..23
hours_future = np.arange(0, forecast_horizon)

plt.figure(figsize=(10, 4))
plt.plot(hours_future, actual_sample, 'o-', label='Actual Future Price')
plt.plot(hours_future, forecast_sample, 'o-', label='Forecasted Price')

plt.title('24-Hour Price Forecast vs Actual for TRAIN SET')
plt.xlabel('Hour Relative to Forecast Start')
plt.ylabel('Settlement Point Price ($/MWh)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()